# Week 2 : FinBERT Implementation & Challenges 



## FinBERT sentiment analysis 

In [7]:


from transformers import BertTokenizer, BertForSequenceClassification
import torch
import torch.nn.functional as F

# loading pretrained FinBERT
tokenizer = BertTokenizer.from_pretrained("ProsusAI/finbert")
model = BertForSequenceClassification.from_pretrained (
    "ProsusAI/finbert",
    use_safetensors=True
)
model.eval()

# starting with single financial sentence sentiment 
text = "The company reported a 20% decline in revenue this quarter."

inputs = tokenizer(
    text,
    return_tensors="pt",
    truncation=True,
    padding=True
)
with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits
probabilities = F.softmax(logits, dim=1)
print("Sentiment probabilities [positive, negative, neutral]:")
print(probabilities)


# Long document handling, 512 token limit 
def finbert_long_text_sentiment(text, tokenizer, model, max_tokens=512):
    tokens = tokenizer.encode(text, add_special_tokens=False)

    chunks = [
        tokens[i:i + max_tokens]
        for i in range(0, len(tokens), max_tokens)
    ]
    all_probs = []

    for chunk in chunks:
        inputs = tokenizer.prepare_for_model(
            chunk,
            return_tensors="pt"
        )
        # adding batch dimensions because HuggingFace tokenizer creates batch dimension for single sentences automatically whereas for long text, we are adding 
        
        inputs = {k: v.unsqueeze(0) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=1)
        all_probs.append(probs)

    avg_probs = torch.mean(torch.stack(all_probs), dim=0)
    return avg_probs

# testing long document sentiment 
long_text = """
The company announced its earnings today.
Revenue declined due to rising operational costs.
However, future guidance remains stable.
"""

result = finbert_long_text_sentiment(long_text, tokenizer, model)
print("Average sentiment [Positive, Negative, Neutral]:")
print(result)





Sentiment probabilities [positive, negative, neutral]:
tensor([[0.0090, 0.9754, 0.0156]])
Average sentiment [Positive, Negative, Neutral]:
tensor([[0.0090, 0.9685, 0.0225]])


## Advanced Topic Challenges 


### Challenge 1 : Catastrophic forgetting 
Catastrophic forgetting happens when a model loses earlier knowledge while learning new data.
In FinBERT, this is intentional and beneficial.
Financial texts use common words with domain-specific meanings.
Example: “depression” shifts from a mental health meaning to an economic downturn.
This semantic shift helps FinBERT specialize in financial language.
Hence, catastrophic forgetting becomes a feature, not a bug.



### challenge 2 : the 512 token limit 

In [17]:

# Truncation Strategy 

from transformers import pipeline

document_text = """
The company reported strong revenue growth in the first quarter.
However, rising inflation and supply chain disruptions posed risks.
Management expects volatility in the upcoming fiscal year.
"""

# loading finbert sentiment analysis pipeline 
finbert = pipeline(
    "sentiment-analysis",
    model="ProsusAI/finbert"
)


truncated_text = document_text[:512]           # keeping only the first 512 tokens 
sentiment = finbert(truncated_text)
print(sentiment)


# Hierarchical approach
# here we split the document into multiple 512 token chunks and run finbert seperately on each chunk 
# and then combine the results as average or weighted 

tokens = tokenizer(document_text)

# splitting tokens into chunks of 512
chunk_size = 512
chunks = []
for i in range(0, len(tokens), 512):
    chunk = document_text[i:i+chunk_size]
    chunks.append(chunk)

# finbert on each chunk 
sentiment_scores = []
for chunk in chunks:
    score = finbert(chunk)  # returns positive, negative, neutral scores
    sentiment_scores.append(score)

# combining the sentiment scores across all chunks 

label_map = {"positive": 1, "neutral": 0, "negative": -1}
weighted_scores = []

for s in sentiment_scores:
    label = s[0]["label"]
    confidence = s[0]["score"]
    weighted_scores.append(label_map[label] * confidence)

final_sentiment = sum(weighted_scores) / len(weighted_scores)
print(final_sentiment)


# Longformer/ bigbird strategy 



Device set to use cpu


[{'label': 'negative', 'score': 0.8505281805992126}]
-0.8505281805992126



### challenge 3 : Slanted Triangular learning rates





FinBERT uses Slanted Triangular Learning Rates instead of a constant learning rate.
The learning rate increases linearly at the start (warm-up phase).
This helps gradients align and the model adapt quickly.
After reaching the maximum rate, it decreases linearly.
The decay phase enables stable training and fine-grained convergence.
STLR improves both training stability and performance.